In [99]:
# Imports
import scipy.integrate as integrate
import numpy as np

In [100]:
# Constants
r_earth = 8.33 # kpc
rho_earth = 0.3 # GeV/cm^3
r_s = 0.184 # kpc
rho_s = 24.42 # GeV/cm^3

Define NFW distribution, $r$ as a function of $s$ and $\theta$, and $\Delta \Omega$.

In [101]:
def rho_NFW(r):
	# Returns in GeV/cm^3, but it doesn't return the correct rho calculated at r_earth
	return rho_s * (r_s / r) * (1 + r / r_s) ** (-2)

def r(s, theta):
	# Returns in kpc
	return np.sqrt(r_earth**2 + s**2 - 2*r_earth*s*np.cos(theta))

def deltaOmega(theta_1, theta_2):
	# Returns in steradians
	return 2 * np.pi * (np.cos(theta_1) - np.cos(theta_2))

Define integrand for the $J$ integral, the function $J(\theta)$, the integrand for the $\bar{J}$ integral and the function $\bar{J}(\theta_1, \theta_2)$. They are all dimensionless.

In [102]:
def J_integrand(s, theta, rho_DM):
	return (rho_DM(r(s, theta)) / rho_earth) ** 2 / r_earth

def J(theta, rho_DM):
	integral, _ = integrate.quad(J_integrand, 0, np.inf, args=(theta, rho_DM))
	return integral

def J_avg_integrand(theta, rho_DM):
	return J(theta, rho_DM) * np.sin(theta)

def J_avg(theta_1, theta_2, rho_DM):
	integral, _ = integrate.quad(J_avg_integrand, theta_1, theta_2, args=(rho_DM,))
	return integral * 2 * np.pi / deltaOmega(theta_1, theta_2)

Use previously defined functions to calculate $\bar{J}$ for different distributions.

In [ ]:
# Integrating from 0 gives convergence issues
J_avg_NFW = J_avg(0, np.pi / 180, rho_NFW)

C:\Users\lucaz\AppData\Local\Temp\ipykernel_9896\2604824896.py:5: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  integral, _ = integrate.quad(J_integrand, 0, np.inf, args=(theta, rho_DM))
C:\Users\lucaz\AppData\Local\Temp\ipykernel_9896\2604824896.py:5: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral, _ = integrate.quad(J_integrand, 0, np.inf, args=(theta, rho_DM))


np.float64(269.55049472711636)